<a href="https://colab.research.google.com/github/Vonewman/AI-projects/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chatbots

The most powerful chatbots in the world all **hallucinate** and lack an up-to-date understanding of the real world. GPT-4, for example, cannot answer questions about recent world events, no matter how important they were.

The world of chatbots is frozen in time. Their world exists as a static snapshot of the world as it was within their training data.

A solution to this problem is *retrieval augmentation*. The idea behind this is plug-in a chatbot to an external vector database where they can then access *accurate* and up-to-date and information. Helping us limit hallucinations and answer questions about the latest events. In this example we will see how to do that.

To begin, we must install the prerequisite libraries that we will be using in this notebook.

In [1]:
!pip install -qU \
  langchain==0.0.162 \
  openai==0.27.7 \
  tiktoken==0.4.0 \
  "pinecone-client[grpc]"==2.2.1 \
  pinecone-datasets=='0.5.0rc11'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 770.9/770.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.2/177.2 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 56.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 54.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Data Preparation

The chatbot use-case requires that we store relevant information (called **contexts**) inside the vector database. To make these retrievable we encode them to create vector embeddings. Once these embeddings are indexed we can use them to augment user queries as demonstrated below:

![Chatbot with retrieval augmentation diagram](https://github.com/pinecone-io/examples/blob/master/learn/generation/chatbots/assets/chatbot-diagram.png?raw=true)

### Downloading the Dataset

We will download a pre-embedding dataset from `pinecone-datasets`. Allowing us to skip the embedding and any other preprocessing steps. When working with your own dataset you will need to perform this embedding step but we have prebuilt the embeddings so we can jump right to the action.

In [2]:
import pinecone_datasets

dataset = pinecone_datasets.load_dataset('wikipedia-simple-text-embedding-ada-002')
dataset.head()

,id,values,sparse_values,metadata,blob
0,1-0,"[-0.011254455894231796, -0.01698738895356655, ...",None,None,"{'chunk': 0, 'source': 'https://simple.wikiped..."
1,1-1,"[-0.0015197008615359664, -0.007858820259571075...",None,None,"{'chunk': 1, 'source': 'https://simple.wikiped..."
2,1-2,"[-0.009930099360644817, -0.012211072258651257,...",None,None,"{'chunk': 2, 'source': 'https://simple.wikiped..."
3,1-3,"[-0.011600767262279987, -0.012608098797500134,...",None,None,"{'chunk': 3, 'source': 'https://simple.wikiped..."
4,1-4,"[-0.026462381705641747, -0.016362832859158516,...",None,None,"{'chunk': 4, 'source': 'https://simple.wikiped..."


In [3]:
len(dataset)

283945

A `pinecone-dataset` always contains `id`, `values`, `sparse_values`, `metadata`, and `blob`. All we need are the IDs, vector embeddings (stored in `values`), and some metadata (which is actually stored in `blob`). Let's reformat the dataset ready for adding to Pinecone, and use a smaller subset of the full dataset.

In [4]:
# we drop sparse_values as they are not needed for this example
dataset.documents.drop(['sparse_values', 'metadata'], axis=1, inplace=True)
dataset.documents.rename(columns={'blob': 'metadata'}, inplace=True)
# we will use rows of the dataset up to index 30_000
dataset.documents.drop(dataset.documents.index[30_000:], inplace=True)
len(dataset)

30000

Now we move on to initializing our Pinecone vector database.

## Creating an Index

In [5]:
index_name = 'chatbot-onboarding'

In [6]:
from getpass import getpass
import pinecone

# find API key in console at app.pinecone.io
YOUR_API_KEY = getpass("Pinecone API Key: ")
# find ENV (cloud region) next to API key in console
YOUR_ENV = input("Pinecone environment: ")

pinecone.init(
    api_key=YOUR_API_KEY,
    environment=YOUR_ENV
)

Pinecone API Key: ··········
Pinecone environment: gcp-starter


In [7]:
import time

if index_name not in pinecone.list_indexes():
    # we create a new index
    pinecone.create_index(
        name=index_name,
        metric='cosine',
        dimension=1536,  # 1536 dim of text-embedding-ada-002
        metadata_config={'indexed': ['wiki-id', 'title']}
    )
    # wait a moment for the index to be fully initialized
    time.sleep(1)

In [8]:
index = pinecone.GRPCIndex(index_name)
# wait a moment for the index to be fully initialized
time.sleep(1)

index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.3,
 'namespaces': {'': {'vector_count': 30000}},
 'total_vector_count': 30000}

In [9]:
index.upsert_from_dataframe(dataset.documents, batch_size=100)

sending upsert requests:   0%|          | 0/30000 [00:00<?, ?it/s]

collecting async responses:   0%|          | 0/300 [00:00<?, ?it/s]

upserted_count: 30000

In [10]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.3,
 'namespaces': {'': {'vector_count': 30000}},
 'total_vector_count': 30000}

## Building the Chatbot and Querying

Now that we've build our index we can switch over to LangChain. We need to initialize a LangChain vector store using the same index we just built. For this we will also need a LangChain embedding object, which we initialize like so:

In [11]:
from getpass import getpass
from langchain.embeddings.openai import OpenAIEmbeddings

OPENAI_API_KEY = getpass("OpenAI API Key: ")  # platform.openai.com
model_name = 'text-embedding-ada-002'

embed = OpenAIEmbeddings(
    model=model_name,
    openai_api_key=OPENAI_API_KEY
)

OpenAI API Key: ··········


In [12]:
from langchain.vectorstores import Pinecone

text_field = "text"

# switch back to normal index for langchain
index = pinecone.Index(index_name)

vectorstore = Pinecone(
    index, embed.embed_query, text_field
)

In [13]:
query = "who was Benito Mussolini?"

vectorstore.similarity_search(
    query,  # our search query
    k=3  # return 3 most relevant docs
)

[Document(page_content='Benito Amilcare Andrea Mussolini KSMOM GCTE (29 July 1883 – 28 April 1945) was an Italian politician and journalist. He was also the Prime Minister of Italy from 1922 until 1943. He was the leader of the National Fascist Party.\n\nBiography\n\nEarly life\nBenito Mussolini was named after Benito Juarez, a Mexican opponent of the political power of the Roman Catholic Church, by his anticlerical (a person who opposes the political interference of the Roman Catholic Church in secular affairs) father. Mussolini\'s father was a blacksmith. Before being involved in politics, Mussolini was a newspaper editor (where he learned all his propaganda skills) and elementary school teacher.\n\nAt first, Mussolini was a socialist, but when he wanted Italy to join the First World War, he was thrown out of the socialist party. He \'invented\' a new ideology, Fascism, much out of Nationalist\xa0and Conservative views.\n\nRise to power and becoming dictator\nIn 1922, he took power b

All of these are good, relevant results. But this actually covers only the first few steps of our retrieval augmented chatbot, the **retrieval augmentation** steps. We're still missing the **chatbot** part.

### Creating the Chatbot

Our chatbot will take the query as a question that is to be answered, but the chatbot must answer the question based on the information it is seeing being returned from the `vectorstore`.

To do this we initialize a chat model and `RetrievalQA` object like so:

In [14]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

# chatbot language model
llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model_name='gpt-3.5-turbo',
    temperature=0.0
)
# retrieval augmented pipeline for chatbot
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

In [15]:
qa.run(query)

'Benito Mussolini was an Italian politician and journalist who served as the Prime Minister of Italy from 1922 until 1943. He was the leader of the National Fascist Party and played a significant role in the rise of fascism in Italy. Mussolini established a dictatorship and implemented various policies to consolidate his power. He aimed to create a new Roman Empire and pursued expansionist policies, including the invasion of Abyssinia (now Ethiopia) in 1935. Mussolini was eventually deposed and captured by the Allies in 1943, and he was executed by Italian partisans in 1945.'

In [16]:
from langchain.chains import RetrievalQAWithSourcesChain

qa_with_sources = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

In [17]:
qa_with_sources(query)

{'question': 'who was Benito Mussolini?',
 'answer': "Benito Mussolini was an Italian politician and journalist who served as the Prime Minister of Italy from 1922 until 1943. He was the leader of the National Fascist Party and played a significant role in the rise of fascism in Italy. Mussolini sought to establish Italy as a new Roman Empire and pursued expansionist policies, including the occupation of Abyssinia (Ethiopia) and Albania. He formed an alliance with Adolf Hitler and entered World War II as part of the Axis Powers. However, Mussolini's regime eventually fell, and he was captured and executed by partisans in 1945. \n",
 'sources': 'https://simple.wikipedia.org/wiki/Benito%20Mussolini'}

In [18]:
pinecone.delete_index(index_name)

## Where would we use this?

Chatbots and retrieval augmented LLMs are incredibly prelevant in our world despite their lack of adoption pre-ChatGPT. Retrieval augmentation is being used to improve the performance and reliability of chatbots and the retrieval *with sources* that we demonstrated above is clearly visible in both Google's Bard and Microsoft's Bing AI. Beyond these tech giants, giving LLMs access to up-to-date information is essential for customer service chatbots that must refer to customer data and FAQ docs, for chatbots plugged into fast changing ecommerce databases, and any scenario where up-to-date information is important or helpful.

---